In [1]:
import pandas as pd
import time
import datetime
import matplotlib.pyplot as plt
import numpy as np
from scipy.constants import golden
from sklearn.linear_model import LinearRegression
import ipywidgets as widgets
import uuid
import os
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score

In [2]:
today = datetime.datetime.today()
print(f"{today.year}-{today.month:02d}-{today.day:02d}")

2020-04-01


In [3]:
print(f"https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-{today.year}-{today.month:02d}-{today.day}.xlsx")

https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-2020-04-1.xlsx


In [4]:
df = pd.read_excel(f"https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-{today.year}-{today.month:02d}-{today.day:02d}.xlsx")

In [5]:
df = df[::-1]
df.head()

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2018
8101,2020-03-21,21,3,2020,1,0,Zimbabwe,ZW,ZWE,14439018.0
8100,2020-03-22,22,3,2020,1,0,Zimbabwe,ZW,ZWE,14439018.0
8099,2020-03-23,23,3,2020,0,0,Zimbabwe,ZW,ZWE,14439018.0
8098,2020-03-24,24,3,2020,0,1,Zimbabwe,ZW,ZWE,14439018.0
8097,2020-03-25,25,3,2020,0,0,Zimbabwe,ZW,ZWE,14439018.0


In [6]:
datecol = df.columns[0]
cases = df.columns[4]
country = df.columns[7]
datecol, cases, country

('dateRep', 'cases', 'geoId')

In [7]:
def logarithmic_curve(x, a, c, d):
    return a / (1 + np.exp(-c * (x - d)))

In [8]:
def exp_curve(x, a, b):
    return a*np.exp(b*x)

In [14]:
def covid_graphs(selectedcountry, threshold=10, image_width=5, save_figure=False):
    nrows = 2
    ncols = 2
    fig, axs = plt.subplots(nrows=nrows, ncols=ncols, figsize=(image_width*ncols, image_width*nrows))
    fig.tight_layout(pad=3.0)
    confirmed = df[(df['cases'] >= 1) & (df[country]==selectedcountry)]['cases']
    confirmed_daily = confirmed.values
    confirmed_cumsum = confirmed.cumsum().values
    if confirmed_daily.sum() < 10:
        return 0
    axs[0, 0].plot(np.arange(0, len(confirmed_daily)), confirmed_daily, color='#fc5e03', label='daily')
    axs[0, 0].plot(np.arange(0, len(confirmed_cumsum)), confirmed_cumsum, color='b', label='cumulative')
    axs[0, 0].plot(np.argmax(confirmed_daily), max(confirmed_daily), ".",  markersize=8, color='r', label='Max cases in one day')
    axs[0, 0].set_yscale('linear')
    axs[0, 0].set_xlabel('Days since first infection')
    axs[0, 0].text(0.65, 0.95, f"Total cases: {confirmed_daily.sum()}", transform=axs[0, 0].transAxes)
    axs[0, 0].legend()
    axs[0, 0].set_title('Infections')
    
    axs[0, 1].set_yscale('log')
    axs[0, 1].plot(np.arange(0, len(confirmed_daily)), confirmed_daily, color='#fc5e03', label='daily')
    axs[0, 1].plot(np.arange(0, len(confirmed_cumsum)), confirmed_cumsum, color='b', label='cumulative')
    axs[0, 1].text(0.65, 0.95, f"Total cases: {confirmed_daily.sum()}", transform=axs[0, 1].transAxes)
    axs[0, 1].plot(np.argmax(confirmed_daily), max(confirmed_daily), ".",  markersize=8, color='r', label='Max cases in one day')
    axs[0, 1].set_xlabel('Days since first infection')
    axs[0, 1].legend()
    axs[0, 1].set_title('Infections (logarithmic scale)')
    
    growth_rate = confirmed_daily/df[(df[country] == selectedcountry) & (df['cases'] >= 1)]['cases'].shift().values
    growth_rate_indices = df[(df[country] == selectedcountry) & (df['cases'] >= 1)].index
    growth_ratedf = df[(df[country] == selectedcountry) & (df['cases'] >= 1)]['cases']/df[(df[country] == selectedcountry) & (df['cases'] >= 1)]['cases'].shift()
    rolling_mean = growth_ratedf.rolling(window=3).mean()
    lr = LinearRegression()
    days_gr = [(df.loc[idx, datecol] - df[(df[country] == selectedcountry) & (df['cases'] >= 1)]['dateRep'].min()).days for idx in df[(df[country] == selectedcountry) & (df['cases'] >= 1)].index]
    days_gr = np.array(days_gr)
    lr.fit(days_gr[1:].reshape(-1,1), growth_rate[1:].reshape(-1,1))

    axs[1, 0].plot(np.arange(0, len(growth_rate_indices)), growth_rate, 'g', ls=':', linewidth=1.5, label='daily rate')
    axs[1, 0].plot(np.arange(0, len(rolling_mean.values)), rolling_mean.values, '#fc5e03', linewidth=2, label='3-day average')
    axs[1, 0].plot(np.arange(0, len(days_gr[1:])), lr.predict(days_gr[1:].reshape(-1,1)), label='trend')

    axs[1, 0].set_xticks(np.arange(0, len(df[(df[country] == selectedcountry) & (df['cases'] >= 1)][cases]), 10))
    axs[1, 0].set_xlabel('Days since first infection')
    axs[1, 0].legend(loc='upper right')
    axs[1, 0].set_title('Growth rate')

    if len(confirmed_cumsum) > 10:
        myfunction = logarithmic_curve
        x = np.arange(0, len(confirmed_cumsum))
        try:
            logistic_params, covariance = curve_fit(myfunction, x, confirmed_cumsum)
            a, c, d = logistic_params
        except Exception as e:
            pass

        axs[1, 1].plot(np.arange(0, len(confirmed_cumsum)), confirmed_cumsum, ".", markersize=8, label="Reported cases")

        if c < 0 or d < 0:
            myfunction = exp_curve
            logistic_params, covariance = curve_fit(myfunction, x, confirmed_cumsum)
                    
        confirmed_pred = myfunction(np.arange(0, len(confirmed_cumsum)), *logistic_params)
        r2 = r2_score(confirmed_cumsum, confirmed_pred)
        axs[1, 1].text(0.2, 0.3, f"$R^2$={r2:.4f}", transform=axs[1, 1].transAxes)
        
        if myfunction == logarithmic_curve:
            confirmed_now = confirmed_cumsum[-1]
            confirmed_then = confirmed_cumsum[-2]
            days = 0
            now = len(confirmed_cumsum)-1
            while confirmed_now - confirmed_then > threshold:
                days += 1
                confirmed_then = confirmed_now
                confirmed_now = logarithmic_curve(now + days, a, c, d)


            x = np.arange(0, len(days_gr) + days)
            y = myfunction(x, a, c, d)
            axs[1, 1].plot([len(confirmed_daily), len(confirmed_daily)], [0, max(y)], color='k', ls=':')
            axs[1, 1].plot(x, y, color='#fc5e03', label="Fitted logistic function")
            axs[1, 1].text(0.55, 0.1, f"Total predicted cases: {int(confirmed_now)}", transform=axs[1, 1].transAxes)
            axs[1, 1].text(0.55, 0.05, f"Remainig growth days: {days}", transform=axs[1, 1].transAxes)
            axs[1, 1].fill_between(x[-days:], confirmed_cumsum[-1], y[-days:],
                                   facecolor="none", hatch="/", edgecolor="b", linewidth=0.0,
                                  label='Predicted increase')

        elif myfunction == exp_curve:
            x = np.arange(0, int(len(confirmed_cumsum)*1.5))
            y = myfunction(x, *logistic_params)
            axs[1, 1].plot(x, y, color='#fc5e03', label="Fitted exponential function")
    
    axs[1, 1].set_xlabel('Days since first infection')
    axs[1, 1].legend(loc='upper left')
    axs[1, 1].set_title('Evolution (prediction)')

    
    country_name = df[df[country] == selectedcountry]['countriesAndTerritories'].values[0]
    plt.suptitle(f"COVID-19 evolution in {country_name}", y=1.025, size=15)
    
    if save_figure:
        filename = f"{selectedcountry}_{today.year}-{today.month:02d}-{today.day:02d}_{uuid.uuid4().hex}.png"
        destination_folder = os.path.expanduser(r"~\Documents\COVID-19")
        plt.savefig(os.path.join(destination_folder, filename),
                    dpi=300, bbox_inches='tight')
        print(f"file saved as: {os.path.join(destination_folder, filename)}")


In [15]:
countries = list(zip(df['countriesAndTerritories'].unique()[::-1], df[country].unique()[::-1]))

In [16]:
output = widgets.interact(covid_graphs,
                 selectedcountry=widgets.Dropdown(options=countries,
                                                  value='ES',
                                                  description='Country:'),
                 threshold=widgets.IntSlider(10, 5, 500, 5), 
                 image_width=widgets.IntSlider(6, 3, 10, 1),
                 save_figure=widgets.Checkbox(False))

interactive(children=(Dropdown(description='Country:', index=174, options=(('Afghanistan', 'AF'), ('Albania', …